Curation by Oksana Bilous.

Accesion: https://figshare.com/s/139f64b495dea9d88c70

In [1]:
!mkdir mcfarland2020
!wget https://figshare.com/ndownloader/articles/10298696?private_link=139f64b495dea9d88c70 -O mcfarland2020/mixseq.zip
!unzip mcfarland2020/mixseq.zip -d mcfarland2020

--2022-03-30 13:30:55--  https://figshare.com/ndownloader/articles/10298696?private_link=139f64b495dea9d88c70
Resolving figshare.com (figshare.com)... 52.30.189.251, 52.48.213.168, 2a05:d018:1f4:d000:9767:1911:7029:1844, ...
Connecting to figshare.com (figshare.com)|52.30.189.251|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2257978361 (2.1G) [application/zip]
Saving to: ‘mcfarland2020/mixseq.zip’

mcfarland2020/mixse 100%[===================>]   2.10G  23.3MB/s    in 43s     

2022-03-30 13:31:38 (50.4 MB/s) - ‘mcfarland2020/mixseq.zip’ saved [2257978361/2257978361]

Archive:  mcfarland2020/mixseq.zip
 extracting: mcfarland2020/README.txt  
 extracting: mcfarland2020/Supplementary Tables.xlsx  
 extracting: mcfarland2020/Trametinib_6hr_expt1.zip  
 extracting: mcfarland2020/Trametinib_24hr_expt1.zip  
 extracting: mcfarland2020/Bortezomib_6hr_expt1.zip  
 extracting: mcfarland2020/Bortezomib_24hr_expt1.zip  
 extracting: mcfarland2020/Idasanutlin_6hr_expt1.

In [2]:
raw_data_path = 'mcfarland2020'

Process files.

In [3]:
import pandas as pd
import numpy as np
import scanpy as sc
from zipfile import ZipFile 
import os
import anndata as ad

In [4]:
sc.logging.print_header()

2022-03-30 13:32:08.111777: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-30 13:32:08.111812: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


scanpy==1.8.2 anndata==0.7.6 umap==0.5.2 numpy==1.20.3 scipy==1.5.3 pandas==1.3.4 scikit-learn==1.0.2 statsmodels==0.11.1 python-igraph==0.8.3 leidenalg==0.8.3 pynndescent==0.5.5


In [5]:
class MIX_seq_data_loader:
    def __init__(self, raw_data_path, expt_no):
        self.raw_data_path = raw_data_path
        self.expt_no = expt_no
#         self.scale = scale
#         self.drugs_SMILES_representation = {
#             'Idasanutlin': 'O=C(O)C1=CC(OC)=C(NC([C@H]2[C@H](C3=C(F)C(Cl)=CC=C3)[C@](C4=CC=C(Cl)C=C4F)(C#N)[C@H](CC(C)(C)C)N2)=O)C=C1',
#             'Bortezomib': 'CC(C)C[C@H](NC(=O)[C@H](CC1=CC=CC=C1)NC(=O)C1=CN=CC=N1)B(O)O',
#             'Navitoclax': 'CC1(CCC(=C(C1)CN2CCN(CC2)C3=CC=C(C=C3)C(=O)NS(=O)(=O)C4=CC(=C(C=C4)NC(CCN5CCOCC5)CSC6=CC=CC=C6)S(=O)(=O)C(F)(F)F)C7=CC=C(C=C7)Cl)C',
#             'Dabrafenib': 'CC(C)(C)C1=NC(=C(S1)C2=NC(=NC=C2)N)C3=C(C(=CC=C3)NS(=O)(=O)C4=C(C=CC=C4F)F)F',
#             'Afatinib': 'CN(C)CC=CC(=O)NC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC(=C(C=C3)F)Cl)OC4CCOC4',
#             'Prexasertib': 'COC1=C(C(=CC=C1)OCCCN)C2=CC(=NN2)NC3=NC=C(N=C3)C#N.Cl.Cl',
#             'Taselisib' : 'CC1=NN(C(=N1)C2=CN3CCOC4=C(C3=N2)C=CC(=C4)C5=CN(N=C5)C(C)(C)C(=O)N)C(C)C',
#             'Gemcitabine': 'C1=CN(C(=O)N=C1N)C2C(C(C(O2)CO)O)(F)F',
#             'Trametinib': 'CC1=C2C(=C(N(C1=O)C)NC3=C(C=C(C=C3)I)F)C(=O)N(C(=O)N2C4=CC=CC(=C4)NC(=O)C)C5CC5',
#             'Everolimus': '[H][C@@]1(C[C@@H](C)[C@]2([H])CC(=O)[C@H](C)\C=C(C)\[C@@H](O)[C@@H](OC)C(=O)[C@H](C)C[C@H](C)\C=C\C=C\C=C(C)\[C@H](C[C@]3([H])CC[C@@H](C)[C@@](O)(O3)C(=O)C(=O)N3CCCC[C@@]3([H])C(=O)O2)OC)CC[C@@H](OCCO)[C@@H](C1)OC',
#             'AZD5591': 'CN1N=C2CSCC3=NN(C)C(CSC4=CC5=C(C=CC=C5)C(OCCCC5=C(N(C)C6=C5C=CC(Cl)=C6C2=C1C)C(O)=O)=C4)=C3',
#             'JQ1': 'ClC1=CC=C(C(C2=C(N3C4=NN=C3C)SC(C)=C2C)=N[C@H]4CC(O)=O)C=C1',
#             'BRD3379': 'CC(=O)NC1=CC=C(C=C1)C(=O)NC2=C(C=C(C=C2)F)N',
#             'DMSO': 'CS(=O)C',
#         }
        self.adata = self.load_adata_obj()
#         self.adata_preprocessing()
        self.adata_add_additional_information()
        
#         self.perturbations = list(self.adata.obs['condition'].unique())
#         self.timetpoints = list(self.adata.obs['timepoint'].unique())
#         self.tissue_types = list(self.adata.obs['tissue_type'].unique())
#         self.drugs_without_SMILES_encoding = list(self.adata.obs[(self.adata.obs['drug_SMILES']=='') & 
#                        (self.adata.obs['condition']!='Untreated')]['condition'].unique())
#         if print_summary:
#             self.print_summary()
    
#     def print_summary(self):
#         print("Experiment", self.expt_no, '\n')
#         print("Perturbations:", self.perturbations, '\n')
#         print("Timetpoints:", self.timetpoints, '\n')
#         print("Tissue types:", self.tissue_types, '\n')
#         print("Number of cells:", len(self.adata.obs), '\n')
#         print("Drugs for which no SMILES encoding was found:", self.drugs_without_SMILES_encoding, '\n')
        
    def load_adata_obj_from_single_file(self, filename):
        zf = ZipFile(self.raw_data_path + '/' + filename)
        zf.extractall(path=self.raw_data_path) 

        df_classifications = pd.read_csv(
            zf.open(zf.namelist()[0] + 'classifications.csv'),
            sep=",",
            index_col = 'barcode'
        )

        adata = sc.read_10x_mtx(self.raw_data_path + '/' + zf.namelist()[0], var_names='gene_symbols')# cache=True)
#         adata.var_names_make_unique()
        adata.obs = df_classifications
        
        if 'hash_tag' in adata.obs.columns:
            adata = adata[~adata.obs["hash_tag"].isin(['multiplet', 'unknown']), :]
            adata.obs['condition'] = adata.obs.apply(lambda x: x['hash_tag'][0:x['hash_tag'].find('_')], axis = 1)
            adata.obs['condition'] = adata.obs.apply(lambda x: 'Trametinib' if x['condition'] == 'Tram' else x['condition'], axis = 1)
            adata.obs['timepoint'] = adata.obs.apply(lambda x: x['hash_tag'][x['hash_tag'].find('_')+1:x['hash_tag'].find('hr')], axis = 1)
        else:
            adata.obs['condition'] = filename[0:filename.find('_')]
            if 'hr' in filename:
                adata.obs['timepoint'] = filename[filename.find('_')+1:filename.find('hr')]
            else:
                adata.obs['timepoint'] = ''

        adata.obs['control'] = adata.obs.apply(lambda x: 'ctrl' if x['condition'] in ['Untreated', 'DMSO']
                                      else 'drug', axis=1)
        adata.obs['cell_line'] = adata.obs.apply(lambda x: x['singlet_ID'][0:x['singlet_ID'].find('_')], axis = 1)
        adata.obs['tissue_type'] = adata.obs.apply(lambda x: x['singlet_ID'][x['singlet_ID'].find('_')+1:], axis = 1)
        
        return(adata)
    
    def load_adata_obj(self):
        files_to_load = []
        for filename in os.listdir(self.raw_data_path):
            if ('.zip' in filename) and ('expt'+str(self.expt_no)+'.' in filename):
                files_to_load.append(filename)

        adata = ad.AnnData()
        for file in files_to_load:
            adata_temp = self.load_adata_obj_from_single_file(file)
            adata_temp.obs['experiment'] = self.expt_no
            if len(adata) == 0:
                adata = adata_temp
            else:
                adata = adata.concatenate(adata_temp)
        if 'batch' in adata.obs.columns:
            adata.obs = adata.obs.drop(columns=['batch'])
            
        return(adata)
    
#     def adata_preprocessing(self):
#         # Filtering out the low-quality cells before doing the analysis
#         self.adata = self.adata[self.adata.obs["cell_quality"] == "normal", :] 

#         # Normalization
#         sc.pp.normalize_total(self.adata, target_sum=1e4)
#         sc.pp.log1p(self.adata)

#         # # Identifying top 5000 highly variable genes
#         # sc.pp.highly_variable_genes(self.adata, n_top_genes = 5000)

#         # Saving the raw data object
#         self.adata.raw = self.adata
        
#         # Scaling
#         if self.scale:
#             sc.pp.scale(self.adata, max_value=10)

#         n_comps = len(self.adata.obs['singlet_ID'].unique()) * 2 
#         # PCA
#         sc.tl.pca(self.adata, svd_solver='arpack', n_comps = n_comps)

#         # UMAP embedding
#         sc.pp.neighbors(self.adata, n_neighbors=10, n_pcs=n_comps)
#         sc.tl.umap(self.adata)
        
    def adata_add_additional_information(self):
        mutant_cls = ['LNCAPCLONEFGC_PROSTATE','DKMG_CENTRAL_NERVOUS_SYSTEM', 'NCIH226_LUNG', 'RCC10RGB_KIDNEY', 'SNU1079_BILIARY_TRACT', 'CCFSTTG1_CENTRAL_NERVOUS_SYSTEM','COV434_OVARY']
        self.adata.obs['TP53_status'] = ['TP53_MUT' if x in mutant_cls else 'TP53_WT' for x in self.adata.obs.singlet_ID.values]
        if self.adata.obs['timepoint'][1] != '':
            self.adata.obs['condition_time'] = self.adata.obs.apply(lambda x: x['condition'] + '_' + x['timepoint'] + 'hr', axis=1)
            
        # Cell Cycle Analysis
        # Loading additional file with lists of s genes and g2m genes
#         cell_phases_df = pd.read_excel(self.raw_data_path+'/Cell_phases.xlsx', sheet_name = 'Cell_phases')
#         s_genes = [x.replace(' ', '') for x in list(cell_phases_df[pd.notna(cell_phases_df.iloc[:,0])].iloc[:,0])]
#         g2m_genes = [x.replace(' ', '') for x in list(cell_phases_df[pd.notna(cell_phases_df.iloc[:,1])].iloc[:,1])]

        # Defining the cell cycles
#         sc.tl.score_genes_cell_cycle(self.adata, s_genes = s_genes, g2m_genes = g2m_genes)

#         self.adata.obs['phase'] = self.adata.obs.apply(lambda x: 'G2/M' if x['phase'] == 'G2M' else ('G0/G1' if x['phase']=='G1' else x['phase']
#         ), axis = 1)
        
#         self.adata.obs['drug_SMILES'] = self.adata.obs.apply(lambda x: self.drugs_SMILES_representation[x['condition']] 
#                                            if x['condition'] in self.drugs_SMILES_representation.keys() else '', axis=1)

In [7]:
%%time
adatas = []
for expt in [1,2,3,5,10]:
    print(expt)
    adatas.append(MIX_seq_data_loader(
        raw_data_path=raw_data_path, 
        expt_no=expt
    ).adata)
adatas

[AnnData object with n_obs × n_vars = 24500 × 32738
     obs: 'singlet_ID', 'num_SNPs', 'singlet_dev', 'singlet_dev_z', 'singlet_margin', 'singlet_z_margin', 'doublet_z_margin', 'tot_reads', 'doublet_dev_imp', 'doublet_CL1', 'doublet_CL2', 'percent.mito', 'cell_det_rate', 'cell_quality', 'doublet_GMM_prob', 'DepMap_ID', 'condition', 'timepoint', 'control', 'cell_line', 'tissue_type', 'experiment', 'TP53_status', 'condition_time'
     var: 'gene_ids',
 AnnData object with n_obs × n_vars = 28165 × 32738
     obs: 'singlet_ID', 'num_SNPs', 'singlet_dev', 'singlet_dev_z', 'singlet_margin', 'singlet_z_margin', 'doublet_z_margin', 'tot_reads', 'doublet_dev_imp', 'doublet_CL1', 'doublet_CL2', 'percent.mito', 'cell_det_rate', 'cell_quality', 'doublet_GMM_prob', 'DepMap_ID', 'condition', 'timepoint', 'control', 'cell_line', 'tissue_type', 'experiment', 'TP53_status'
     var: 'gene_ids',
 AnnData object with n_obs × n_vars = 72326 × 32738
     obs: 'singlet_ID', 'num_SNPs', 'singlet_dev', 'sing

In [8]:
adata = ad.concat(adatas, keys=[n for n in [1,2,3,5,10]], label='experiment', index_unique='--', merge='same')
adata

AnnData object with n_obs × n_vars = 180400 × 32738
    obs: 'singlet_ID', 'num_SNPs', 'singlet_dev', 'singlet_dev_z', 'singlet_margin', 'singlet_z_margin', 'doublet_z_margin', 'tot_reads', 'doublet_dev_imp', 'doublet_CL1', 'doublet_CL2', 'percent.mito', 'cell_det_rate', 'cell_quality', 'doublet_GMM_prob', 'DepMap_ID', 'condition', 'timepoint', 'control', 'cell_line', 'tissue_type', 'experiment', 'TP53_status'
    var: 'gene_ids'

In [9]:
!rm -r mcfarland2020

In [10]:
adata.write(f'Mcfarland_2020_raw.h5ad')

/home/icb/yuge.ji/miniconda3/envs/py37/lib/python3.7/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'singlet_ID' as categorical
/home/icb/yuge.ji/miniconda3/envs/py37/lib/python3.7/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'doublet_CL1' as categorical
/home/icb/yuge.ji/miniconda3/envs/py37/lib/python3.7/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and